In [1]:
import os,warnings,gc
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from sklearn.metrics import roc_auc_score,auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedGroupKFold,train_test_split
import lightgbm as lgb
import xgboost as xgb
import catboost as cgb

I supressed outputs cuz, I ended up with a OOM exception after submitting. You can uncomment it if you need.

In [2]:
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(4)

polars.config.Config

In [3]:
path_to_train = "/home/sohail/Downloads/credit_risk/train/"
path_to_test = "/home/sohail/Downloads/credit_risk/test/"

In [4]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""/home/sohail/D…","""train_applprev…"
13,"""/home/sohail/D…","""train_applprev…"
…,…,…
7,"""/home/sohail/D…","""train_base.par…"
11,"""/home/sohail/D…","""train_credit_b…"


index,path,filename
i64,str,str
31,"""/home/sohail/D…","""test_applprev_…"
35,"""/home/sohail/D…","""test_applprev_…"
…,…,…
1,"""/home/sohail/D…","""test_applprev_…"
33,"""/home/sohail/D…","""test_base.parq…"


In [5]:
train_base = (
    pl.read_parquet(path_to_train+"/train_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("WEEK_NUM").cast(pl.UInt8).alias("weeknum"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.weekday().alias("weekday"),
        pl.col("Date").dt.week().alias("week"),
        (pl.col("Date").dt.year() - 2018).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
test_base = (
    pl.read_parquet(path_to_test+"/test_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("WEEK_NUM").cast(pl.UInt8).alias("weeknum")
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.weekday().alias("weekday"),
        pl.col("Date").dt.week().alias("week"),
        (pl.col("Date").dt.year() - 2018).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_case_ids = train_base["case_id"].to_numpy()
test_case_ids = test_base["case_id"].to_numpy()
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ],
        how="vertical"
    )
)
del train_base,test_base
gc.collect()
total_base

case_id,Date,weeknum,month,weekday,week,year,target
u32,date,u8,i8,i8,i8,i32,u8
0,2019-01-03,0,1,4,1,1,0
1,2019-01-03,0,1,4,1,1,0
…,…,…,…,…,…,…,…
57633,2020-10-06,92,10,2,41,2,0
57634,2020-10-06,92,10,2,41,2,0


In [6]:
def reduce_dtypes(df:pl.DataFrame,name):
    if name == "intshallow":
        df = df.rename(
            {
                "dpdmaxdatemonth_442T":"dpdmaxdatemonth_442D",
                "dpdmaxdatemonth_89T":"dpdmaxdatemonth_89D",
                "dpdmaxdateyear_596T":"dpdmaxdateyear_596D",
                "dpdmaxdateyear_896T":"dpdmaxdateyear_896D",
                "overdueamountmaxdatemonth_284T":"overdueamountmaxdatemonth_284D",
                "overdueamountmaxdatemonth_365T":"overdueamountmaxdatemonth_365D",
                "overdueamountmaxdateyear_2T":"overdueamountmaxdateyear_2D",
                "overdueamountmaxdateyear_994T":"overdueamountmaxdateyear_994D",
            }
        )
    elif name == "intdepth":
        df = df.rename(
            {
                "pmts_month_158T":"pmts_month_158D",
                "pmts_month_706T":"pmts_month_706D",
                "pmts_year_1139T":"pmts_year_1139D",
                "pmts_year_507T":"pmts_year_507D",
            }
        )
    elif name == "extshallow":
        df = df.rename(
            {
                "dpdmaxdatemonth_804T":"dpdmaxdatemonth_804D",
                "dpdmaxdateyear_742T":"dpdmaxdateyear_742D",
                "overdueamountmaxdatemonth_494T":"overdueamountmaxdatemonth_494D",
                "overdueamountmaxdateyear_432T":"overdueamountmaxdateyear_432D",
            }
        )
    return (
        df
        .select(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("T","M").cast(pl.String),
            cs.ends_with("P","A").cast(pl.Float32),
            (cs.ends_with("L") & cs.numeric()).cast(pl.Float32)
        )
    )

def grouping(df:pl.DataFrame):
    return (
        df
        .group_by("case_id")
        .agg(
            ((cs.ends_with("P","A","D")) | (cs.ends_with("L") & cs.numeric())).mean().prefix("mean_"),
            ((cs.ends_with("P","A","D")) | (cs.ends_with("L") & cs.numeric())).max().prefix("max_"),
            # ((cs.ends_with("P","A","D")) | (cs.ends_with("L") & cs.numeric())).min().prefix("min_"),
            # ((cs.ends_with("P","A")) | (cs.ends_with("L") & cs.numeric())).var().prefix("var_"),
            (cs.string() | cs.boolean()).drop_nulls().mode().first().prefix("mode_")
        )
    )

def date_features(df:pl.DataFrame):
    return (
        df
        .join(
            total_base
            .select(["case_id","Date"]),
            on="case_id",
            how="left"
        )
        .select(
            pl.col("case_id"),
            (pl.col(pl.Date) - pl.col("Date")).dt.total_days(),
            (pl.col(pl.Date).exclude("Date").dt.year() - 2018).cast(pl.Int8).prefix("year_"),
            pl.col(pl.Date).exclude("Date").dt.month().cast(pl.Int8).prefix("month_"),
            pl.col(pl.Date).exclude("Date").dt.week().cast(pl.Int16).prefix("week_"),
            # pl.col(pl.Date).exclude("Date").dt.weekday().cast(pl.Int16).prefix("weekday_"),
            (~(cs.date() | cs.datetime())).exclude("case_id").prefix("nodate_")
        )
        .drop("Date")
    )

def select_low_cat_cols(df:pl.DataFrame,thresh=200):
    cols = []
    for col_name in df.select(pl.col(pl.Categorical)).columns:
        if df.select(pl.col(col_name).unique()).shape[0] >= thresh:
            cols.append(col_name)
        if df.select(pl.col(col_name).unique()).shape[0] <= 1:
            cols.append(col_name)
    return df.drop(cols)

def select_impuatable(df:pl.DataFrame,thresh=0.95):
    cols =  (
        df
        .select(pl.all().is_null().mean())
        .transpose(include_header=True)
        .filter(pl.col("column_0") < thresh)
        ["column"].to_list()
    )
    return df.select(cols)

def preprocess(filter_string:str,prefix_string:str):
    train_files_list = train_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    test_files_list = test_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    with pl.StringCache():
        train_df = (
            pl.concat(
                [
                    pl.read_parquet(_).pipe(reduce_dtypes,prefix_string).pipe(grouping) for _ in train_files_list
                ]
            )
        )
        test_df = (
            pl.concat(
                [
                    pl.read_parquet(_).pipe(reduce_dtypes,prefix_string).pipe(grouping) for _ in test_files_list
                ]
            )
        )
    return (
        pl.concat(
            [
                train_df,
                test_df
            ],
            how="vertical_relaxed",
            rechunk=True
        )
        .pipe(select_impuatable)
        .pipe(select_low_cat_cols)
        # .pipe(date_features)
        .with_columns(
            pl.col(pl.String).cast(pl.Categorical).rank("dense")
        )
        .select(pl.col("case_id"),pl.all().shrink_dtype().exclude("case_id").prefix(f"{prefix_string}_"))
    )


In [7]:
total_past_shallow = preprocess("applprev_1", "pastshallow")
total_past_shallow

case_id,pastshallow_mean_approvaldate_319D,pastshallow_mean_creationdate_885D,pastshallow_mean_dateactivated_425D,pastshallow_mean_dtlastpmt_581D,pastshallow_mean_dtlastpmtallstes_3545839D,pastshallow_mean_employedfrom_700D,pastshallow_mean_firstnonzeroinstldate_307D,pastshallow_mean_actualdpd_943P,pastshallow_mean_annuity_853A,pastshallow_mean_credacc_actualbalance_314A,pastshallow_mean_credacc_credlmt_575A,pastshallow_mean_credacc_maxhisbal_375A,pastshallow_mean_credacc_minhisbal_90A,pastshallow_mean_credamount_590A,pastshallow_mean_currdebt_94A,pastshallow_mean_downpmt_134A,pastshallow_mean_mainoccupationinc_437A,pastshallow_mean_maxdpdtolerance_577P,pastshallow_mean_outstandingdebt_522A,pastshallow_mean_revolvingaccount_394A,pastshallow_mean_byoccupationinc_3656910L,pastshallow_mean_childnum_21L,pastshallow_mean_credacc_transactions_402L,pastshallow_mean_pmtnum_8L,pastshallow_mean_tenor_203L,pastshallow_max_approvaldate_319D,pastshallow_max_creationdate_885D,pastshallow_max_dateactivated_425D,pastshallow_max_dtlastpmt_581D,pastshallow_max_dtlastpmtallstes_3545839D,pastshallow_max_employedfrom_700D,pastshallow_max_firstnonzeroinstldate_307D,pastshallow_max_actualdpd_943P,pastshallow_max_annuity_853A,pastshallow_max_credacc_actualbalance_314A,pastshallow_max_credacc_credlmt_575A,pastshallow_max_credacc_maxhisbal_375A,pastshallow_max_credacc_minhisbal_90A,pastshallow_max_credamount_590A,pastshallow_max_currdebt_94A,pastshallow_max_downpmt_134A,pastshallow_max_mainoccupationinc_437A,pastshallow_max_maxdpdtolerance_577P,pastshallow_max_outstandingdebt_522A,pastshallow_max_revolvingaccount_394A,pastshallow_max_byoccupationinc_3656910L,pastshallow_max_childnum_21L,pastshallow_max_credacc_transactions_402L,pastshallow_max_pmtnum_8L,pastshallow_max_tenor_203L,pastshallow_mode_cancelreason_3545846M,pastshallow_mode_district_544M,pastshallow_mode_education_1138M,pastshallow_mode_postype_4733339M,pastshallow_mode_profession_152M,pastshallow_mode_rejectreason_755M,pastshallow_mode_rejectreasonclient_4145042M
u32,date,date,date,date,date,date,date,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,date,date,date,date,date,date,date,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u16,u8,u8,u16,u8,u8
159535,2018-01-31,2018-01-31,2018-02-08,2017-06-23,2018-08-11,null,2018-03-03,0.00,5815.60,null,0.00,null,null,43890.00,8295.25,0.00,33000.00,0.00,8744.20,null,null,1.00,null,7.00,7.00,2018-12-10,2018-12-10,2018-12-20,2017-06-23,2019-09-30,null,2019-01-09,0.00,8881.00,null,0.00,null,null,80000.00,16590.51,0.00,46000.00,0.00,17488.40,null,null,1.00,null,11.00,11.00,1,1,1,1,1,1,1
1436782,2015-07-29,2016-03-07,2015-08-05,2017-11-19,2017-11-19,null,2015-12-26,0.00,3321.96,null,0.00,null,null,22790.70,0.00,64.00,24833.33,0.25,0.00,null,15250.00,0.25,null,9.90,9.90,2018-03-05,2018-03-05,2018-03-09,2019-01-28,2019-01-28,null,2018-04-02,0.00,5259.20,null,0.00,null,null,40000.00,0.00,640.00,50000.00,1.00,0.00,null,40000.00,1.00,null,24.00,24.00,1,2,2,1,1,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
100984,2022-09-12,2022-09-12,null,null,2022-09-20,null,null,0.00,0.00,1816.71,0.00,4000.00,0.00,0.00,0.00,0.00,null,0.00,0.00,840473664.00,null,null,3.00,null,null,2022-09-12,2022-09-12,null,null,2022-09-20,null,null,0.00,0.00,1816.71,0.00,4000.00,0.00,0.00,0.00,0.00,null,0.00,0.00,840473664.00,null,null,3.00,null,null,1,39,2,6,1,1,1
101005,null,2022-09-12,null,null,null,null,2022-10-15,0.00,4699.70,null,0.00,null,null,125000.00,null,0.00,50000.00,null,null,null,null,null,null,54.00,54.00,null,2022-09-13,null,null,null,null,2022-10-15,0.00,5847.00,null,0.00,null,null,150000.00,null,0.00,50000.00,null,null,null,null,null,null,60.00,60.00,2,580,2,4,1,1,2


In [20]:
total_past_shallow.select(~cs.date())

case_id,pastshallow_mean_actualdpd_943P,pastshallow_mean_annuity_853A,pastshallow_mean_credacc_actualbalance_314A,pastshallow_mean_credacc_credlmt_575A,pastshallow_mean_credacc_maxhisbal_375A,pastshallow_mean_credacc_minhisbal_90A,pastshallow_mean_credamount_590A,pastshallow_mean_currdebt_94A,pastshallow_mean_downpmt_134A,pastshallow_mean_mainoccupationinc_437A,pastshallow_mean_maxdpdtolerance_577P,pastshallow_mean_outstandingdebt_522A,pastshallow_mean_revolvingaccount_394A,pastshallow_mean_byoccupationinc_3656910L,pastshallow_mean_childnum_21L,pastshallow_mean_credacc_transactions_402L,pastshallow_mean_pmtnum_8L,pastshallow_mean_tenor_203L,pastshallow_max_actualdpd_943P,pastshallow_max_annuity_853A,pastshallow_max_credacc_actualbalance_314A,pastshallow_max_credacc_credlmt_575A,pastshallow_max_credacc_maxhisbal_375A,pastshallow_max_credacc_minhisbal_90A,pastshallow_max_credamount_590A,pastshallow_max_currdebt_94A,pastshallow_max_downpmt_134A,pastshallow_max_mainoccupationinc_437A,pastshallow_max_maxdpdtolerance_577P,pastshallow_max_outstandingdebt_522A,pastshallow_max_revolvingaccount_394A,pastshallow_max_byoccupationinc_3656910L,pastshallow_max_childnum_21L,pastshallow_max_credacc_transactions_402L,pastshallow_max_pmtnum_8L,pastshallow_max_tenor_203L,pastshallow_mode_cancelreason_3545846M,pastshallow_mode_district_544M,pastshallow_mode_education_1138M,pastshallow_mode_postype_4733339M,pastshallow_mode_profession_152M,pastshallow_mode_rejectreason_755M,pastshallow_mode_rejectreasonclient_4145042M
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u16,u8,u8,u16,u8,u8
159535,0.00,5815.60,null,0.00,null,null,43890.00,8295.25,0.00,33000.00,0.00,8744.20,null,null,1.00,null,7.00,7.00,0.00,8881.00,null,0.00,null,null,80000.00,16590.51,0.00,46000.00,0.00,17488.40,null,null,1.00,null,11.00,11.00,1,1,1,1,1,1,1
1436782,0.00,3321.96,null,0.00,null,null,22790.70,0.00,64.00,24833.33,0.25,0.00,null,15250.00,0.25,null,9.90,9.90,0.00,5259.20,null,0.00,null,null,40000.00,0.00,640.00,50000.00,1.00,0.00,null,40000.00,1.00,null,24.00,24.00,1,2,2,1,1,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
100984,0.00,0.00,1816.71,0.00,4000.00,0.00,0.00,0.00,0.00,null,0.00,0.00,840473664.00,null,null,3.00,null,null,0.00,0.00,1816.71,0.00,4000.00,0.00,0.00,0.00,0.00,null,0.00,0.00,840473664.00,null,null,3.00,null,null,1,39,2,6,1,1,1
101005,0.00,4699.70,null,0.00,null,null,125000.00,null,0.00,50000.00,null,null,null,null,null,null,54.00,54.00,0.00,5847.00,null,0.00,null,null,150000.00,null,0.00,50000.00,null,null,null,null,null,null,60.00,60.00,2,580,2,4,1,1,2


In [26]:
(
    total_past_shallow
    .join(
        total_base
        .select(["case_id","Date"]),
        how="left",
        on="case_id"
    )
    .select(
        pl.col("case_id"),
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days().prefix("tdays_"),
        (pl.col(pl.Date).exclude("Date").dt.year() - 2018).cast(pl.Int8).prefix("year_"),
        pl.col(pl.Date).exclude("Date").dt.month().cast(pl.Int8).prefix("month_"),
        pl.col(pl.Date).exclude("Date").dt.week().cast(pl.Int16).prefix("week_"),
        pl.col(pl.Date).exclude("Date").dt.weekday().cast(pl.Int16).prefix("weekday_"),
        (~cs.date()).exclude("case_id").prefix("notdate_")
    )
    .drop("tdays_Date")
)

case_id,tdays_pastshallow_mean_approvaldate_319D,tdays_pastshallow_mean_creationdate_885D,tdays_pastshallow_mean_dateactivated_425D,tdays_pastshallow_mean_dtlastpmt_581D,tdays_pastshallow_mean_dtlastpmtallstes_3545839D,tdays_pastshallow_mean_employedfrom_700D,tdays_pastshallow_mean_firstnonzeroinstldate_307D,tdays_pastshallow_max_approvaldate_319D,tdays_pastshallow_max_creationdate_885D,tdays_pastshallow_max_dateactivated_425D,tdays_pastshallow_max_dtlastpmt_581D,tdays_pastshallow_max_dtlastpmtallstes_3545839D,tdays_pastshallow_max_employedfrom_700D,tdays_pastshallow_max_firstnonzeroinstldate_307D,year_pastshallow_mean_approvaldate_319D,year_pastshallow_mean_creationdate_885D,year_pastshallow_mean_dateactivated_425D,year_pastshallow_mean_dtlastpmt_581D,year_pastshallow_mean_dtlastpmtallstes_3545839D,year_pastshallow_mean_employedfrom_700D,year_pastshallow_mean_firstnonzeroinstldate_307D,year_pastshallow_max_approvaldate_319D,year_pastshallow_max_creationdate_885D,year_pastshallow_max_dateactivated_425D,year_pastshallow_max_dtlastpmt_581D,year_pastshallow_max_dtlastpmtallstes_3545839D,year_pastshallow_max_employedfrom_700D,year_pastshallow_max_firstnonzeroinstldate_307D,month_pastshallow_mean_approvaldate_319D,month_pastshallow_mean_creationdate_885D,month_pastshallow_mean_dateactivated_425D,month_pastshallow_mean_dtlastpmt_581D,month_pastshallow_mean_dtlastpmtallstes_3545839D,month_pastshallow_mean_employedfrom_700D,month_pastshallow_mean_firstnonzeroinstldate_307D,month_pastshallow_max_approvaldate_319D,…,nodate_pastshallow_mean_credamount_590A,nodate_pastshallow_mean_currdebt_94A,nodate_pastshallow_mean_downpmt_134A,nodate_pastshallow_mean_mainoccupationinc_437A,nodate_pastshallow_mean_maxdpdtolerance_577P,nodate_pastshallow_mean_outstandingdebt_522A,nodate_pastshallow_mean_revolvingaccount_394A,nodate_pastshallow_mean_byoccupationinc_3656910L,nodate_pastshallow_mean_childnum_21L,nodate_pastshallow_mean_credacc_transactions_402L,nodate_pastshallow_mean_pmtnum_8L,nodate_pastshallow_mean_tenor_203L,nodate_pastshallow_max_actualdpd_943P,nodate_pastshallow_max_annuity_853A,nodate_pastshallow_max_credacc_actualbalance_314A,nodate_pastshallow_max_credacc_credlmt_575A,nodate_pastshallow_max_credacc_maxhisbal_375A,nodate_pastshallow_max_credacc_minhisbal_90A,nodate_pastshallow_max_credamount_590A,nodate_pastshallow_max_currdebt_94A,nodate_pastshallow_max_downpmt_134A,nodate_pastshallow_max_mainoccupationinc_437A,nodate_pastshallow_max_maxdpdtolerance_577P,nodate_pastshallow_max_outstandingdebt_522A,nodate_pastshallow_max_revolvingaccount_394A,nodate_pastshallow_max_byoccupationinc_3656910L,nodate_pastshallow_max_childnum_21L,nodate_pastshallow_max_credacc_transactions_402L,nodate_pastshallow_max_pmtnum_8L,nodate_pastshallow_max_tenor_203L,nodate_pastshallow_mode_cancelreason_3545846M,nodate_pastshallow_mode_district_544M,nodate_pastshallow_mode_education_1138M,nodate_pastshallow_mode_postype_4733339M,nodate_pastshallow_mode_profession_152M,nodate_pastshallow_mode_rejectreason_755M,nodate_pastshallow_mode_rejectreasonclient_4145042M
u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u16,u8,u8,u16,u8,u8
159535,-593,-593,-585,-815,-401,null,-562,-280,-280,-270,-815,14,null,-250,0,0,0,-1,0,null,0,0,0,0,-1,1,null,1,1,1,2,6,8,null,3,12,…,43890.00,8295.25,0.00,33000.00,0.00,8744.20,null,null,1.00,null,7.00,7.00,0.00,8881.00,null,0.00,null,null,80000.00,16590.51,0.00,46000.00,0.00,17488.40,null,null,1.00,null,11.00,11.00,1,1,1,1,1,1,1
1436782,-1440,-1218,-1433,-596,-596,null,-1290,-490,-490,-486,-161,-161,null,-462,-3,-2,-3,-1,-1,null,-3,0,0,0,1,1,null,0,7,3,8,11,11,null,12,3,…,22790.70,0.00,64.00,24833.33,0.25,0.00,null,15250.00,0.25,null,9.90,9.90,0.00,5259.20,null,0.00,null,null,40000.00,0.00,640.00,50000.00,1.00,0.00,null,40000.00,1.00,null

In [22]:
5 * 14 + 1 + (58 - 14)

115

In [9]:
115 - 58

57

In [7]:
total_past_shallow = preprocess("applprev_1", "pastshallow")
# display(total_past_shallow)
print("done 1")

total_past_depth = preprocess("applprev_2", "pastdepth")
# display(total_past_depth)
print("done 2")

total_static_base = preprocess("static_0","staticbase")
# display(total_static_base)
print("done 3")


total_static_external = (
    preprocess("static_cb", "staticexternal")
    .with_columns(
        cs.contains("riskassesment_302T").str.split("%").list.gather([0, 1]).apply(
            lambda x: (int(x[0]) + int(x[1].split("-")[1])) / 200
        )
    )
)
# display(total_static_external)
print("done 4")


total_person_shallow = preprocess("person_1", "personshallow")
# display(total_person_shallow)
print("done 5")


total_person_depth = preprocess("person_2", "persondepth")
# display(total_person_depth)
print("done 6")


total_other_shallow = preprocess("other_1", "othershallow")
# display(total_other_shallow)
print("done 7")


total_deposit_shallow = preprocess("deposit_1", "depositshallow")
# display(total_deposit_shallow)
print("done 8")


total_debitcard_shallow = preprocess("debitcard", "cardshallow")
# display(total_debitcard_shallow)
print("done 9")


total_credit_internal_shallow = preprocess("bureau_a_1", "intshallow")
# display(total_credit_internal_shallow)
print("done 10")


total_credit_internal_depth = preprocess("bureau_a_2", "intdepth")
# display(total_credit_internal_depth)
print("done 11")


total_credit_external_shallow = preprocess("bureau_b_1", "extshallow")
# display(total_credit_external_shallow)
print("done 12")


total_credit_external_depth = preprocess("bureau_b_2", "extdepth")
# display(total_credit_external_depth)
print("done 13")


total_registry_a = preprocess("registry_a", "rega")
# display(total_registry_a)
print("done 14")


total_registry_b = preprocess("registry_b", "regb")
# display(total_registry_b)
print("done 15")


total_registry_c = preprocess("registry_c", "regc")
# display(total_registry_c)
print("done 16")

done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9
done 10
done 11
done 12
done 13
done 14
done 15
done 16


In [8]:
train_total = (
    total_base
    .join(
        total_past_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_past_depth,
        on="case_id",
        how="left"
    )
    .join(
        total_static_base,
        on="case_id",
        how="left"
    )
    .join(
        total_static_external,
        on="case_id",
        how="left"
    )
    .join(
        total_person_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_person_depth,
        on="case_id",
        how="left"
    )
    .join(
        total_other_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_depth,
        on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow,
        on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth,
        on="case_id",
        how="left"
    )
    .join(
        total_registry_a,
        on="case_id",
        how="left"
    )
    .join(
        total_registry_b,
        on="case_id",
        how="left"
    )
    .join(
        total_registry_c,
        on="case_id",
        how="left"
    )
    .drop("Date")
    .filter(
        pl.col("case_id").is_in(train_case_ids)
    )
    .drop("case_id")
    .select(pl.all().shrink_dtype())
)
train_total

weeknum,month,weekday,week,year,target,pastshallow_mean_approvaldate_319D,pastshallow_mean_creationdate_885D,pastshallow_mean_dateactivated_425D,pastshallow_mean_dtlastpmt_581D,pastshallow_mean_dtlastpmtallstes_3545839D,pastshallow_mean_employedfrom_700D,pastshallow_mean_firstnonzeroinstldate_307D,pastshallow_max_approvaldate_319D,pastshallow_max_creationdate_885D,pastshallow_max_dateactivated_425D,pastshallow_max_dtlastpmt_581D,pastshallow_max_dtlastpmtallstes_3545839D,pastshallow_max_employedfrom_700D,pastshallow_max_firstnonzeroinstldate_307D,pastshallow_year_mean_approvaldate_319D,pastshallow_year_mean_creationdate_885D,pastshallow_year_mean_dateactivated_425D,pastshallow_year_mean_dtlastpmt_581D,pastshallow_year_mean_dtlastpmtallstes_3545839D,pastshallow_year_mean_employedfrom_700D,pastshallow_year_mean_firstnonzeroinstldate_307D,pastshallow_year_max_approvaldate_319D,pastshallow_year_max_creationdate_885D,pastshallow_year_max_dateactivated_425D,pastshallow_year_max_dtlastpmt_581D,pastshallow_year_max_dtlastpmtallstes_3545839D,pastshallow_year_max_employedfrom_700D,pastshallow_year_max_firstnonzeroinstldate_307D,pastshallow_month_mean_approvaldate_319D,pastshallow_month_mean_creationdate_885D,pastshallow_month_mean_dateactivated_425D,…,rega_year_mean_recorddate_4527225D,rega_year_max_recorddate_4527225D,rega_month_mean_recorddate_4527225D,rega_month_max_recorddate_4527225D,rega_week_mean_recorddate_4527225D,rega_week_max_recorddate_4527225D,rega_weekday_mean_recorddate_4527225D,rega_weekday_max_recorddate_4527225D,rega_nodate_mean_amount_4527230A,rega_nodate_max_amount_4527230A,rega_nodate_mode_name_4527232M,regb_mean_deductiondate_4917603D,regb_max_deductiondate_4917603D,regb_year_mean_deductiondate_4917603D,regb_year_max_deductiondate_4917603D,regb_month_mean_deductiondate_4917603D,regb_month_max_deductiondate_4917603D,regb_week_mean_deductiondate_4917603D,regb_week_max_deductiondate_4917603D,regb_weekday_mean_deductiondate_4917603D,regb_weekday_max_deductiondate_4917603D,regb_nodate_mean_amount_4917619A,regb_nodate_max_amount_4917619A,regb_nodate_mode_name_4917606M,regc_mean_processingdate_168D,regc_max_processingdate_168D,regc_year_mean_processingdate_168D,regc_year_max_processingdate_168D,regc_month_mean_processingdate_168D,regc_month_max_processingdate_168D,regc_week_mean_processingdate_168D,regc_week_max_processingdate_168D,regc_weekday_mean_processingdate_168D,regc_weekday_max_processingdate_168D,regc_nodate_mean_pmtamount_36A,regc_nodate_max_pmtamount_36A,regc_nodate_mode_employername_160M
u8,i8,i8,i8,i8,u8,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,f32,f32,u32,i16,i16,i8,i8,i8,i8,i8,i8,i8,i8,f32,f32,u16,i16,i16,i8,i8,i8,i8,i8,i8,i8,i8,f32,f32,u32
0,1,4,1,1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0,1,4,1,1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
91,10,1,41,2,0,-1379,-1521,-1375,-722,-480,null,-1644,-292,-292,-287,-293,4,null,-261,-2,-2,-2,0,1,null,-2,1,1,1,1,2,null,2,12,8,12,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
91,10,1,41,2,0,-269,-26

In [9]:
del train_files_df
del test_files_df
del total_base
del total_past_shallow
del total_past_depth
del total_static_base
del total_static_external
del total_person_depth
del total_person_shallow
del total_other_shallow
del total_deposit_shallow
del total_debitcard_shallow
del total_credit_external_depth
del total_credit_external_shallow
del total_credit_internal_depth
del total_credit_internal_shallow
del total_registry_a
del total_registry_b
del total_registry_c
gc.collect()

0

In [10]:
class_params = {
    "objective":"binary",
    "boosting_type": "gbdt",
    "metric":"auc",
    "max_depth": 128,
    "learning_rate": 0.01,
    "n_estimators": 10000,
    "colsample_bynode": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 420,
    "reg_alpha": 0.15,
    "reg_lambda": 15,
    "extra_trees": True,
    "num_leaves": 256,
    "device": "gpu",
    "importance_type": "gain",
    "verbose": -1
}

xgb_params = {
    "objective": "binary:logistic",
    "n_estimators": 10000,
    "eval_metric": "auc",
    "seed": 420,
    "booster": "gbtree",
    "device": "cuda",
    "eta": 0.01,
    "gamma": 5,
    "max_depth": 128,
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "lambda": 10,
    "alpha": 2,
    "updater": "grow_gpu_hist",
    "grow_policy": "depthwise",
    "max_leaves": 256,
    "num_parallel_tree": 1,    
    "sample_type": "uniform",
    "normalize_type": "tree",
    "rate_drop": 0.15,
    "skip_drop": 0.9,
    "enable_categorical": True
}

cat_params = {
    "eval_metric": "AUC",
    "task_type": "GPU",
    "iterations": 10000,
    "learning_rate": 0.01,
    "bootstrap_type": "Poisson",
    "random_seed": 420,
    "l2_leaf_reg": 15,
    "subsample": 0.8,
    "depth": 32,
    "max_leaves": 64,
    "grow_policy": "Lossguide",
    "od_type": "Iter",
    "od_wait": 1000,
    "verbose": 10
}

In [11]:
week_num = train_total["weeknum"]
train_total = train_total.drop("weeknum")
gc.collect()

def filter_ind(df:pl.DataFrame,indexes:np.array):
    return (
        df
        .with_row_index()
        .filter(
            pl.col("index").is_in(indexes)
        )
        .drop("index")
    )

In [12]:
gc.collect()

0

In [13]:
num_splits = 5
cv = StratifiedGroupKFold(n_splits=num_splits,shuffle=False)
lgb_clf_models = [
    lgb.LGBMClassifier(**class_params) for _ in range(num_splits)
]
xgb_clf_models = [
    xgb.XGBClassifier(**xgb_params) for _ in range(num_splits)
]
cat_models = [
    cgb.CatBoostClassifier(**cat_params) for _ in range(num_splits)
]
for i,(train_ind,valid_ind) in enumerate(
    cv.split(train_total.drop("target"),
    train_total.select("target"),
    groups=week_num)
    ):
    print(f"Training start for LGBClassifier: {i+1}")
    lgb_clf_models[i].fit(
        (
            train_total
            .drop("target")
            .pipe(filter_ind,train_ind)
        ),
        (
            train_total
            .select("target")
            .pipe(filter_ind,train_ind)
        ),
        eval_set=[(
            (
                train_total
                .pipe(filter_ind,valid_ind)
            ),
            (
                train_total
                .select("target")
                .pipe(filter_ind,valid_ind)
            )
        )],
        callbacks=[lgb.log_evaluation(10),lgb.early_stopping(1000)]
    )
    print(gc.collect())
    print(f"Training start for XGBClassifier: {i+1}")
    early_stop = xgb.callback.EarlyStopping(rounds=1000)
    log_eval = xgb.callback.EvaluationMonitor(period=10)
    xgb_clf_models[i].fit(
        (
            train_total
            .drop("target")
            .pipe(filter_ind,train_ind)
        ),
        (
            train_total
            .select("target")
            .pipe(filter_ind,train_ind)
        ),
        eval_set=[(
            (
                train_total
                .drop("target")
                .pipe(filter_ind,valid_ind)
            ),
            (
                train_total
                .select("target")
                .pipe(filter_ind,valid_ind)
            )
        )],
        callbacks=[early_stop,log_eval],
        verbose=False
    )
    print(gc.collect())
    print(f"Training start for CatBoostClassifier: {i+1}")
    cat_models[i].fit(
        (
            train_total
            .drop("target")
            .pipe(filter_ind,train_ind)
            .to_pandas()
        ),
        (
            train_total
            .select("target")
            .pipe(filter_ind,train_ind)
            .to_pandas()
        ),
        eval_set=[(
            (
                train_total
                .drop("target")
                .pipe(filter_ind,valid_ind)
                .to_pandas()
            ),
            (
                train_total
                .select("target")
                .pipe(filter_ind,valid_ind)
                .to_pandas()
            )
        )]
    )
    print(gc.collect())

Training start for LGBClassifier: 1
Training until validation scores don't improve for 1000 rounds
[10]	valid_0's auc: 0.677015
[20]	valid_0's auc: 0.663521
[30]	valid_0's auc: 0.664297
[40]	valid_0's auc: 0.669632
[50]	valid_0's auc: 0.674551
[60]	valid_0's auc: 0.67826
[70]	valid_0's auc: 0.680364
[80]	valid_0's auc: 0.682004
[90]	valid_0's auc: 0.681495
[100]	valid_0's auc: 0.68193
[110]	valid_0's auc: 0.683248
[120]	valid_0's auc: 0.686015
[130]	valid_0's auc: 0.686755
[140]	valid_0's auc: 0.689198
[150]	valid_0's auc: 0.691364
[160]	valid_0's auc: 0.69049
[170]	valid_0's auc: 0.690259
[180]	valid_0's auc: 0.689915
[190]	valid_0's auc: 0.690022
[200]	valid_0's auc: 0.691171
[210]	valid_0's auc: 0.692005
[220]	valid_0's auc: 0.691143
[230]	valid_0's auc: 0.691185
[240]	valid_0's auc: 0.691595
[250]	valid_0's auc: 0.692278
[260]	valid_0's auc: 0.692344
[270]	valid_0's auc: 0.693816
[280]	valid_0's auc: 0.693688
[290]	valid_0's auc: 0.694066
[300]	valid_0's auc: 0.693896
[310]	valid_0

In [ ]:
# class Model:

#     def __init__(
#             self,
#             lgb_clf_models:list,
#             xgb_clf_models:list,
#             cat_models:list
#             ) -> None:
#         self.lgb_clf_models = lgb_clf_models
#         self.xgb_clf_models = xgb_clf_models
#         self.cat_models =  cat_models
    
#     def predict_proba(self,X):
#         self.lgb_clf_pred = [_.predict_proba(
#             X
#             # .select(lgb_cols)
#             )[:,1] for _ in self.lgb_clf_models]
#         self.xgb_clf_pred = [_.predict_proba(
#             X
#             # .select(xgb_cols)
#             )[:,1] for _ in self.xgb_clf_models]
#         self.cat_pred = [_.predict_proba(
#             X
#             # .select(cat_cols)
#             .to_pandas()
#             )[:,1] for _ in self.cat_models]

#         return np.mean(
#             self.lgb_clf_pred+\
#             self.xgb_clf_pred+\
#             self.cat_pred,
#             axis=0
#             )

In [ ]:
# model = Model(
#     lgb_clf_models,
#     xgb_clf_models,
#     cat_models
#     )

In [ ]:
lgb_imp = pd.DataFrame(index=train_total.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])
for _ in range(5):
    for imp,col_name in sorted(zip(lgb_clf_models[_].feature_importances_,train_total.drop("target").columns)):
        lgb_imp.loc[col_name,f"gain_{_+1}"] = imp
lgb_imp.reset_index(inplace=True)
lgb_imp.to_parquet("/home/sohail/Downloads/lgb_imp_mean.parquet")

lgb_imp = pd.DataFrame(index=train_total.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])
for _ in range(5):
    for imp,col_name in sorted(zip(xgb_clf_models[_].feature_importances_,train_total.drop("target").columns)):
        lgb_imp.loc[col_name,f"gain_{_+1}"] = imp
lgb_imp.reset_index(inplace=True)
lgb_imp.to_parquet("/home/sohail/Downloads/xgb_imp_mean.parquet")

lgb_imp = pd.DataFrame(index=train_total.drop("target").columns,columns=["gain_1","gain_2","gain_3","gain_4","gain_5"])
for _ in range(5):
    for imp,col_name in sorted(zip(cat_models[_].feature_importances_,train_total.drop("target").columns)):
        lgb_imp.loc[col_name,f"gain_{_+1}"] = imp
lgb_imp.reset_index(inplace=True)
lgb_imp.to_parquet("/home/sohail/Downloads/cat_imp_mean.parquet")

In [ ]:
# model.lgb_clf_pred

In [ ]:
# gini_stability(valid_df,np.mean(model.lgb_clf_pred,axis=0))

In [ ]:
# model.xgb_clf_pred

In [ ]:
# gini_stability(valid_df,np.mean(model.xgb_clf_pred,axis=0))

In [ ]:
# model.cat_pred

In [ ]:
# gini_stability(valid_df,np.mean(model.cat_pred,axis=0))